In [ ]:
pip install nltk

In [ ]:
pip install seaborn

In [ ]:
pip install scikit-learn

In [ ]:
pip install wordcloud

In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import nltk
import seaborn as sns
import string
from wordcloud import WordCloud
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score
import pickle

In [3]:
df = pd.read_csv('spam.csv', encoding='latin-1')

In [4]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [12]:
df.shape

(5572, 5)

In [13]:
# 1. data cleaning 
# 2. EDA
# 3. Text preprocessing 
# 4. Model bulding 
# 5. Evaluation 
# 6. Impeovement 
# 7. Website
# 8. Deployment 

## 1. Data Cleaning 

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [15]:
#remove columns 
df.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'], inplace = True)

In [16]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [17]:
#Renaming columns 
df.rename(columns={'v1' : 'target', 'v2' : 'text'}, inplace = True)
df.head()

,target,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [21]:
spam_messages = df[df['target'].str.contains('spam', case=False)]


In [27]:
spam_messages

,target,text
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
5,spam,FreeMsg Hey there darling it's been 3 week's n...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...
11,spam,"SIX chances to win CASH! From 100 to 20,000 po..."
...,...,...
5537,spam,Want explicit SEX in 30 secs? Ring 02073162414...
5540,spam,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...
5547,spam,Had your contract mobile 11 Mnths? Latest Moto...
5566,spam,REMINDER FROM O2: To get 2.50 pounds free call...


In [ ]:
encoder = LabelEncoder()
df['target'] = encoder.fit_transform(df['target'])

In [ ]:
df.head()

In [ ]:
# missing values
df.isnull().sum()

In [ ]:
# Check for duplicates values
df.duplicated().sum()

In [ ]:
# Remove duplicates
df = df.drop_duplicates(keep= 'first')

In [ ]:
df.duplicated().sum()

In [ ]:
df.shape

## 2. EDA

In [ ]:
df.head()

In [ ]:
plt.pie(df['target'].value_counts(), labels=['ham','spam'], autopct = '%0.2f')
plt.show()

In [ ]:
# Data imbalanced

In [ ]:
df['num_characters'] = df['text'].apply(len)

In [ ]:
df['num_words'] = df['text'].apply(lambda x:len(nltk.word_tokenize(x)))

In [ ]:
df.head()

In [ ]:
df['num_sent']=df['text'].apply(lambda x:len(nltk.sent_tokenize(x)))

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
# Ham messages Decs
df[df['target'] == 0][['num_characters','num_words','num_sent']].describe()

In [ ]:
# Spam messages Decs
df[df['target'] == 1][['num_characters','num_words','num_sent']].describe()

In [ ]:
plt.figure(figsize = (15,10))
sns.histplot(df[df['target'] == 0]['num_characters'])
sns.histplot(df[df['target'] == 1]['num_characters'],color = 'red')

In [ ]:
plt.figure(figsize = (15,10))
sns.histplot(df[df['target'] == 0]['num_words'])
sns.histplot(df[df['target'] == 1]['num_words'])

In [ ]:
plt.figure(figsize = (15,10))
sns.histplot(df[df['target'] == 0]['num_sent'])
sns.histplot(df[df['target'] == 1]['num_sent'])

In [ ]:
# most of spam messages have less number of characters, words and sentenses 

In [ ]:
sns.pairplot(df, hue = 'target')

In [ ]:
corr = df[['target','num_characters','num_words','num_sent']]

In [ ]:
sns.heatmap(corr.corr(), annot= True)

In [ ]:
# It seems like there is multicollinearity in the variables, so we will proceed with only one column.

## Text Preprocessing 

In [ ]:
# 1. Lower Case
# 2. Tokenization
# 3. Removing Special Characters
# 4. Removing Stop Words and Punctuation
# 5. Lemmatization

In [ ]:
from nltk.corpus import stopwords
stopwords.words('english')

In [ ]:
string.punctuation

In [ ]:
from nltk.stem.porter import PorterStemmer

In [ ]:
ps = PorterStemmer()

In [ ]:
ps.stem('dancing')

In [ ]:
from nltk.stem import WordNetLemmatizer

# Initialize the WordNet lemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
nltk.download()

In [ ]:
lemmatizer.lemmatize('loving')

In [ ]:
def transform_text(text):
    text = text.lower()
    text = nltk.word_tokenize(text)

    y =[]
    for i in text:
        if i.isalnum():
            y.append(i)

    text = y[:]
    y.clear()
    for i in text:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)

    text = y[:]
    y.clear()
    for i in text:
        y.append(ps.stem(i))


    return " ".join(y)
            

In [ ]:
transform_text('Is that seriously how you spell his name?')

In [ ]:
df['text'][20]

In [ ]:
df['transform_text'] = df['text'].apply(transform_text)

In [ ]:
df.head()

In [ ]:
wc = WordCloud(width=500,height=500,min_font_size=10,background_color='white')
span_wc = wc.generate(df[df['target'] ==1]['transform_text'].str.cat(sep=" "))
plt.figure(figsize=(15,6))
plt.imshow(span_wc)

In [ ]:

hamp_wc = wc.generate(df[df['target'] ==0]['transform_text'].str.cat(sep=" "))
plt.figure(figsize=(15,6))
plt.imshow(span_wc)

In [ ]:
spam_corpus = []
for message in df[df['target']==1]['transform_text'].tolist():
    for words in message.split():
        spam_corpus.append(words)
        

In [ ]:
len(spam_corpus)

In [ ]:
sns.barplot(x=pd.DataFrame(Counter(spam_corpus).most_common(30))[0],y=pd.DataFrame(Counter(spam_corpus).most_common(30))[1])
plt.xticks(rotation = 90)
plt.show()

In [ ]:
ham_corpus = []
for message in df[df['target']==0]['transform_text'].tolist():
    for words in message.split():
        ham_corpus.append(words)

In [ ]:
len(ham_corpus)

In [ ]:
sns.barplot(x=pd.DataFrame(Counter(ham_corpus).most_common(30))[0],y=pd.DataFrame(Counter(ham_corpus).most_common(30))[1])
plt.xticks(rotation = 90)
plt.show()

## 4.Model Building

In [ ]:
cv = CountVectorizer()

tfidf = TfidfVectorizer(max_features=3000)

In [ ]:
X = tfidf.fit_transform(df['transform_text']).toarray()

In [ ]:
X

In [ ]:
y = df['target']. values

In [ ]:
y

In [ ]:
X_train,X_test,y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=2)

In [ ]:
gnb =  GaussianNB()
mnb = MultinomialNB()
bnb = BernoulliNB()

In [ ]:
gnb.fit(X_train,y_train)
y_pred = gnb.predict(X_test)
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(precision_score(y_test,y_pred))

In [ ]:
mnb.fit(X_train,y_train)
y_pred1 = mnb.predict(X_test)
print(accuracy_score(y_test,y_pred1))
print(confusion_matrix(y_test,y_pred1))
print(precision_score(y_test,y_pred1))

In [ ]:
bnb.fit(X_train,y_train)
y_pred2 = bnb.predict(X_test)
print(accuracy_score(y_test,y_pred2))
print(confusion_matrix(y_test,y_pred2))
print(precision_score(y_test,y_pred2))

In [ ]:
pickle.dump(tfidf,open('vectorizer.pkl','wb'))
pickle.dump(mnb,open('model.pkl','wb'))
            

In [ ]:
# Pipeline 
#1. Preprocessing (tranform)
#2. Vecterize
#3. perform algoritham 